In [1]:
# Imports
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal
import os

print("All imports successful")

c:\Users\Nababa\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
c:\Users\Nababa\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All imports successful


In [2]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")

print("✅ API key loaded")

✅ API key loaded


In [3]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,  # Lower temperature for more precise tool usage
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


In [4]:
# Tools: calculator, weather, dictionary, web search
from langchain_core.tools import tool
import os
import requests

@tool
def weather_tool(location: str) -> str:
    """Return current weather for a location.

    - If `OPENWEATHER_API_KEY` is set in the environment, this will call OpenWeatherMap.
    - Otherwise it returns a placeholder message and instructions.
    """
    api_key = os.getenv("OPENWEATHER_API_KEY")
    if api_key:
        try:
            url = "http://api.openweathermap.org/data/2.5/weather"
            params = {"q": location, "appid": api_key, "units": "metric"}
            r = requests.get(url, params=params, timeout=10)
            r.raise_for_status()
            data = r.json()
            desc = data["weather"][0]["description"]
            temp = data["main"]["temp"]
            return f"{location}: {desc}, {temp}°C"
        except Exception as e:
            return f"Error fetching weather: {e}"
    else:
        return (f"No OpenWeatherMap API key found. To enable real lookups, set the environment variable OPENWEATHER_API_KEY.\n"
                f"Placeholder for {location}: clear sky, 25°C")

@tool
def dictionary_tool(word: str) -> str:
    """Lookup a word definition using the free Dictionary API (dictionaryapi.dev).

    Falls back to a friendly message if the network call fails.
    """
    try:
        r = requests.get(f"https://api.dictionaryapi.dev/api/v2/entries/en/{word}", timeout=10)
        if r.status_code == 200:
            data = r.json()
            meanings = data[0].get("meanings", [])
            if meanings:
                defs = meanings[0].get("definitions", [])
                if defs:
                    definition = defs[0].get("definition", "")
                    example = defs[0].get("example", "")
                    return (f"{word}: {definition}" + (f" Example: {example}" if example else ""))
        return f"No definition found for {word} (status {r.status_code})"
    except Exception as e:
        return f"Error fetching definition: {e}"

@tool
def web_search(query: str, num_results: int = 3) -> str:
    """Perform a simple web search and return top result titles/links.

    This uses DuckDuckGo's HTML endpoint and requires `beautifulsoup4` for parsing.
    If `beautifulsoup4` is not installed or parsing fails, returns a short message.
    """
    try:
        r = requests.post("https://html.duckduckgo.com/html/", data={"q": query}, timeout=10)
        r.raise_for_status()
        try:
            from bs4 import BeautifulSoup
        except Exception:
            return ("Search performed but `beautifulsoup4` is required to parse results. "
                    "Install it with `pip install beautifulsoup4`.")
        soup = BeautifulSoup(r.text, "html.parser")
        results = []
        for el in soup.select(".result__a")[:num_results]:
            title = el.get_text().strip()
            href = el.get('href')
            results.append(f"{title} — {href}")
        if results:
            return "\n".join(results)
        return "No results found."
    except Exception as e:
        return f"Error performing web search: {e}\nNote: web search requires network access and `beautifulsoup4`."

print(" Tools:  weather_tool, dictionary_tool, web_search created")

 Tools:  weather_tool, dictionary_tool, web_search created


In [5]:
# Create a list of tools
tools = [weather_tool, web_search, dictionary_tool]

# Bind tools to the LLM
llm_with_tools = llm.bind_tools(tools)

print(f"✅ LLM bound to {len(tools)} tools")
print(f"   Tools: {[tool.name for tool in tools]}")

✅ LLM bound to 3 tools
   Tools: ['weather_tool', 'web_search', 'dictionary_tool']


In [ ]:
# Tests: run each tool and a quick LLM-with-tools check
print("=== Tools quick test ===")

print("\n-- Calculator --")
try:
    print("234 * 567 =>", calculator("234 * 567"))
except Exception as e:
    print("Calculator test failed:", e)

print("\n-- Dictionary --")
try:
    print("serendipity =>", dictionary_tool("serendipity"))
except Exception as e:
    print("Dictionary test failed:", e)

print("\n-- Weather --")
try:
    print("New York =>", weather_tool("New York"))
except Exception as e:
    print("Weather test failed:", e)

print("\n-- Web Search (top 2) --")
try:
    print(web_search("Python 3.14 release", num_results=2))
except Exception as e:
    print("Web search test failed:", e)

print("\n-- LLM with tools check --")
try:
    # If `llm_with_tools` is bound above, ask a simple arithmetic question to see if it uses the calculator
    if 'llm_with_tools' in globals():
        resp = llm_with_tools.invoke([HumanMessage(content="What is 234 * 567?")])
        print("LLM response content:\n", getattr(resp, 'content', resp))
        print("LLM tool_calls:\n", getattr(resp, 'tool_calls', None))
    else:
        print("`llm_with_tools` not found. Run the LLM binding cell first.")
except Exception as e:
    print("LLM-with-tools test failed:", e)

print("\n=== End of tests ===")
